In [1]:
import pandas as pd
import re,string
import nltk
from patsy import dmatrices
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize, word_tokenize
import matplotlib.pyplot as plt
import warnings
%pylab inline
import numpy as np
from sklearn.manifold import MDS
from sklearn.metrics import euclidean_distances

Populating the interactive namespace from numpy and matplotlib


In [3]:
google_data = pd.read_csv('google1.csv')

In [4]:
google_data.head()

,Title,Date Written,Rating,Current/Former,Job Title,Location,Recommendation?,Outlook,Main Text,Pros,Cons,Advice to management
0,"""Operations manager""","Oct 5, 2018",5.0,Current Employee,Operations Manager,"New York, NY",Recommends,Positive Outlook,I have been working at Google full-time (More ...,"Perks, food, spaces, culture, health care and ...","Flat org, hard to grow",NaN
1,"""Moving at the speed of light, burn out is ine...","Jun 21, 2013",4.0,Former Employee,Program Manager,"Mountain View, CA",Doesn't Recommend,Negative Outlook,Show More,"1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,NaN
2,"""Great balance between big-company security an...","May 10, 2014",5.0,Current Employee,Software Engineer III,"New York, NY",Recommends,Positive Outlook,Show More,"* If you're a software engineer, you're among ...","* It *is* becoming larger, and with it comes g...",NaN
3,"""The best place I've worked and also the most ...","Feb 8, 2015",5.0,Current Employee,Anonymous Employee,"Mountain View, CA",Recommends,Positive Outlook,Show More,You can't find a more well-regarded company th...,I live in SF so the commute can take between 1...,NaN
4,"""Unique, one of a kind dream job""","Jul 19, 2018",5.0,Former Employee,Software Engineer,"Los Angeles, CA",Recommends,Neutral Outlook,Roll back the trend towards becoming a regular...,Google is a world of its own. At every other c...,"If you don't work in MTV (HQ), you will be giv...",NaN


### Step 1: pros/cons data pre-processing

In [8]:
#clean punctuation and white spaces
google_data["pros_clean"] = google_data.Pros.apply(lambda x:re.sub(r'[^\w\s]', ' ', x.lower()))
google_data["pros_clean"] = google_data.pros_clean.apply(lambda x:re.sub(r'(\\n+)', ' ', x))
google_data["pros_clean"] = google_data.pros_clean.apply(lambda x:re.sub(r'^b[\'\"\'"\"''\s]', '', x.lower()))
google_data["pros_clean"] = google_data.pros_clean.apply(lambda x:re.sub(r'\s+', ' ', x))

google_data["cons_clean"] = google_data.Cons.apply(lambda x:re.sub(r'[^\w\s]', ' ', x.lower()))
google_data["cons_clean"] = google_data.cons_clean.apply(lambda x:re.sub(r'(\\n+)', ' ', x))
google_data["cons_clean"] = google_data.cons_clean.apply(lambda x:re.sub(r'^b[\'\"\'"\"''\s]', '', x.lower()))
google_data["cons_clean"] = google_data.cons_clean.apply(lambda x:re.sub(r'\s+', ' ', x))

#tokenizing and removing stop words
stop = set(stopwords.words('english'))
punc = string.punctuation
google_data['pros_clean'] = google_data.pros_clean.apply(lambda x: [word for word in word_tokenize(x) if word not in stop])
google_data['pros_clean'] = google_data['pros_clean'].apply(lambda x: [word for word in x if word not in punc])
google_data['cons_clean'] = google_data.cons_clean.apply(lambda x: [word for word in word_tokenize(x) if word not in stop])
google_data['cons_clean'] = google_data['cons_clean'].apply(lambda x: [word for word in x if word not in punc])

google_data.head()

,Title,Date Written,Rating,Current/Former,Job Title,Location,Recommendation?,Outlook,Main Text,Pros,Cons,Advice to management,pros_clean,cons_clean
0,"""Operations manager""","Oct 5, 2018",5.0,Current Employee,Operations Manager,"New York, NY",Recommends,Positive Outlook,I have been working at Google full-time (More ...,"Perks, food, spaces, culture, health care and ...","Flat org, hard to grow",NaN,"[perks, food, spaces, culture, health, care, w...","[flat, org, hard, grow]"
1,"""Moving at the speed of light, burn out is ine...","Jun 21, 2013",4.0,Former Employee,Program Manager,"Mountain View, CA",Doesn't Recommend,Negative Outlook,Show More,"1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,NaN,"[1, food, food, food, 15, cafes, main, campus,...","[1, work, life, balance, balance, perks, benef..."
2,"""Great balance between big-company security an...","May 10, 2014",5.0,Current Employee,Software Engineer III,"New York, NY",Recommends,Positive Outlook,Show More,"* If you're a software engineer, you're among ...","* It *is* becoming larger, and with it comes g...",NaN,"[software, engineer, among, kings, hill, googl...","[becoming, larger, comes, growing, pains, bure..."
3,"""The best place I've worked and also the most ...","Feb 8, 2015",5.0,Current Employee,Anonymous Employee,"Mountain View, CA",Recommends,Positive Outlook,Show More,You can't find a more well-regarded company th...,I live in SF so the commute can take between 1...,NaN,"[find, well, regarded, company, actually, dese...","[live, sf, commute, take, 1, 5, hours, 1, 75, ..."
4,"""Unique, one of a kind dream job""","Jul 19, 2018",5.0,Former Employee,Software Engineer,"Los Angeles, CA",Recommends,Neutral Outlook,Roll back the trend towards becoming a regular...,Google is a world of its own. At every other c...,"If you don't work in MTV (HQ), you will be giv...",NaN,"[google, world, every, company, lots, people, ...","[work, mtv, hq, given, work, however, passiona..."


### Step 2: get pros/cons attributes

In [10]:
#get POS
get_pros = google_data['pros_clean'].apply(lambda x: [word for word in nltk.pos_tag(x)])
get_cons = google_data['cons_clean'].apply(lambda x: [word for word in nltk.pos_tag(x)])

pro_adj = []
con_adj = []

def get_adj(empty_list,pos_data):
    for review in pos_data:
        for word,pos in review:
            if pos == 'JJ' or pos == 'JJR' or pos == 'JJS': # if the POS-tag is adjective
                empty_list.append(word)
get_adj(pro_adj,get_pros)
get_adj(con_adj,get_cons)

In [13]:
from collections import Counter
Counter(pro_adj).most_common(30)

[('great', 2998),
 ('good', 1789),
 ('smart', 768),
 ('free', 724),
 ('best', 501),
 ('nice', 351),
 ('many', 264),
 ('awesome', 217),
 ('excellent', 199),
 ('salary', 198),
 ('flexible', 180),
 ('new', 177),
 ('interesting', 170),
 ('open', 160),
 ('friendly', 160),
 ('high', 156),
 ('amazing', 139),
 ('fantastic', 124),
 ('google', 123),
 ('big', 118),
 ('innovative', 104),
 ('easy', 98),
 ('competitive', 91),
 ('different', 90),
 ('incredible', 85),
 ('top', 84),
 ('cool', 83),
 ('overall', 83),
 ('large', 80),
 ('huge', 80)]

In [15]:
Counter(con_adj).most_common(30)

[('big', 689),
 ('many', 543),
 ('hard', 521),
 ('much', 423),
 ('good', 361),
 ('large', 327),
 ('difficult', 323),
 ('great', 292),
 ('long', 262),
 ('bad', 227),
 ('little', 227),
 ('high', 195),
 ('new', 177),
 ('competitive', 174),
 ('political', 146),
 ('slow', 138),
 ('small', 134),
 ('internal', 131),
 ('best', 124),
 ('huge', 122),
 ('corporate', 114),
 ('easy', 113),
 ('middle', 111),
 ('google', 106),
 ('low', 105),
 ('smart', 104),
 ('bureaucratic', 101),
 ('salary', 96),
 ('different', 93),
 ('poor', 93)]

In [16]:
#manually took some attributes from the top frequent pros words
pros_attribute = ['great','good','happy','nice','decent','excellent','best','ethical','strong','flexible','new',
                  'easy','friendly','positive','different','professional','high','solid','corporate',
                  'smart','stable','large']

In [17]:
#manually took some attributes from the top frequent cons words
cons_attribute = ['low','little','hard','difficult','long','poor','limited','bad','slow','terrible','conservative'
                 ,'horrible','different','less','senior','political','bureaucratic']

### Step 3: lemmentize -> get replacement

In [18]:
#create a function that would return WORDNET POS compliance to WORDENT lemmatization (a,n,r,v) 
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
         return wordnet.ADV
    else:
        # As default pos in lemmatization is Noun
        return wordnet.NOUN
    
wnl = WordNetLemmatizer()

#Lemmentization
def wn_pos(filtered_pos,empty_list):
    for word,pos in filtered_pos:
        empty_list.append(wnl.lemmatize(word,get_wordnet_pos(pos)))
        #print pos
        #print get_wordnet_pos(pos)
    return empty_list

In [21]:
pros_lem = []
cons_lem = []

def get_lem(emptylist, pos_data):
    for review in pos_data:
        for word,pos in review:
            wn_pos(review,emptylist)
get_lem(pros_lem, get_pros)
get_lem(cons_lem,get_cons)

In [22]:
pros_words = nltk.FreqDist(pros_lem)
cons_words = nltk.FreqDist(cons_lem)

rslt_unique_pros = pd.DataFrame.from_dict(pros_words,orient='index').reset_index()
rslt_unique_pros.columns = ['word','frequency']

rslt_unique_cons = pd.DataFrame.from_dict(cons_words,orient='index').reset_index()
rslt_unique_cons.columns = ['word','frequency']

In [28]:
replacement = {'work_life_balance':['time','life','balance','sabbatical','sabbaticals','focus','hour','day','health','flexible','week'
                                    ,'vacation','schedule','overtime'],
                'culture_value':['people','culture','team','care','value','product','coworkers','atmosphere','competitive'
                                ,'family','collaboration','respect','community','colleague','supportive','vision','diversity'],
                'career_oppotunity':['opportunity','learn','industry','career','license','training','train','growth','grow'
                                    ,'level','position','development','advancement','advance','study','build','skill','resource'
                                    ,'education','potential'],
                'company_benefit':['company','benefit','pay','financial','financially','provide','salary','bonus','offer'
                                  ,'401k','package','stock','compensation','invest','investment','money','performance','reward'
                                  ,'retirement','promote','insurance'],
                'senior_management':['place','environment','management','help','manager','experience','match'
                                                ,'plan','office','support','location','leadership','treat','helpful','senior'
                                                ,'manage','leader','communication']}

In [29]:
def getKeysByValue(dictOfElements, valueToFind):
    for k,v  in dictOfElements.items():
        if valueToFind in v:
            return(k)
    return  valueToFind

def replace_attributes(s):
    return([getKeysByValue(replacement,y) for y in s])

In [30]:
google_data['pros_replace'] = google_data['pros_clean'].map(replace_attributes)
google_data['cons_replace'] = google_data['cons_clean'].map(replace_attributes)

### Step 4: Lift Score

In [31]:
def ratio(x,y):
    if x==0:
        return float(y)
    if y==0:
        return float(x)
    return(float(x)*float(y))

def get_lift(a,b,tokenized_data):
    '''Function to calculate lift scores given any two words from a list of tokenized words'''
    if (a==b):
        return 1
    p_a = len([i for i in tokenized_data if a in i])
    p_b = len([i for i in tokenized_data if (b in i)])
    p_a_b = len([i for i in tokenized_data if a in i if b in i])
    n = len(tokenized_data)
    return float(float(n)*float(p_a_b)/ratio(p_a,p_b))

In [34]:
pros_list = replacement.keys()
lift_score = [get_lift(x,y,google_data.pros_replace) for x in pros_attribute for y in pros_list]
formatted_lift_score = [ round(elem,2) for elem in lift_score ]

In [35]:
pd.DataFrame(reshape(formatted_lift_score,(len(pros_attribute),len(pros_list))),index =pros_attribute , columns = pros_list )

,culture_value,company_benefit,senior_management,work_life_balance,career_oppotunity
great,1.30,1.15,1.25,1.08,1.06
good,0.91,1.52,1.24,1.22,0.91
happy,1.14,1.18,1.57,2.11,0.89
nice,1.10,0.93,1.54,0.85,0.70
decent,0.64,2.99,0.71,1.38,1.28
excellent,1.23,1.55,1.10,0.99,1.46
best,0.86,1.34,1.33,1.01,1.42
ethical,0.78,1.25,0.00,0.00,2.44
strong,1.68,2.22,1.23,1.11,2.17
flexible,0.00,0.00,0.00,0.00,0.00


In [36]:
cons_list = replacement.keys()
lift_score = [get_lift(x,y,google_data.cons_replace) for x in cons_attribute for y in cons_list]
formatted_lift_score = [ round(elem,2) for elem in lift_score ]

In [37]:
pd.DataFrame(reshape(formatted_lift_score,(len(cons_attribute),len(cons_list))),index =cons_attribute , columns = cons_list )

,culture_value,company_benefit,senior_management,work_life_balance,career_oppotunity
low,1.39,2.60,1.23,0.81,1.80
little,1.53,1.27,1.70,0.96,1.98
hard,1.27,1.30,0.89,1.50,1.33
difficult,1.27,1.25,1.20,1.48,1.99
long,1.22,0.77,0.99,2.21,1.08
poor,1.79,1.25,3.20,2.29,2.25
limited,0.95,1.03,1.76,0.83,4.28
bad,1.36,1.11,1.85,1.11,0.84
slow,0.72,1.38,0.51,0.74,2.10
terrible,1.82,1.21,2.36,0.65,1.61


### Lift for mission Statement

    for Pros

In [81]:
google_values = ['empowerment', 'micromanagement', 'openness', 'democracy', 'fun', 'freedom']

In [82]:
mission_lift = [get_lift(x,y,google_data.pros_replace) for x in google_values for y in pros_list]
formatted_lift_score = [ round(elem,2) for elem in mission_lift ]

In [83]:
pd.DataFrame(reshape(formatted_lift_score,(len(google_values),len(pros_list))),index =google_values , columns = pros_list )

,culture_value,company_benefit,senior_management,work_life_balance,career_oppotunity
empowerment,1.56,1.25,2.38,2.62,0.00
micromanagement,1.06,1.02,0.97,1.43,1.33
openness,1.56,1.56,1.49,1.64,0.61
democracy,0.00,0.00,0.00,0.00,0.00
fun,1.21,0.92,1.70,1.13,0.88
freedom,0.93,0.82,1.23,0.98,1.14


    for Cons

In [84]:
mission_lift = [get_lift(x,y,google_data.cons_replace) for x in google_values for y in cons_list]
formatted_lift_score = [ round(elem,2) for elem in mission_lift ]

In [85]:
pd.DataFrame(reshape(formatted_lift_score,(len(google_values),len(cons_list))),index =google_values , columns = cons_list )

,culture_value,company_benefit,senior_management,work_life_balance,career_oppotunity
empowerment,0.00,0.00,0.00,0.00,0.00
micromanagement,4.24,1.88,2.20,3.03,1.61
openness,1.32,2.35,2.75,0.00,2.01
democracy,0.00,0.00,0.00,0.00,0.00
fun,2.04,1.32,0.71,1.94,0.83
freedom,1.12,0.74,1.45,1.59,0.42
